In [1]:
import re
import numpy as np
import pandas as pd
import xlwings as xl

from IPython.display import display

In [2]:
df = pd.read_csv("MoodleGrades.csv", index_col=0)

ctg_id_list = df.index.values
grade = df["Grade on Moodle"].values

In [3]:
modules = []
competencies = []
levels = []

for ctg_id in ctg_id_list:
    result = re.search(r"^(\d)\.(\d)\.(\d)\.(\d)$", ctg_id)
    modules.append(result.group(1))
    competencies.append("{}.{}".format(result.group(1), result.group(2)))
    levels.append(result.group(3))
    
modules = np.array(modules).astype(int)
competencies = np.array(competencies).astype(str)
levels = np.array(levels).astype(int)

In [4]:
def create_module_dataframe(module_number):
    in_module = modules == module_number
    module_competencies = np.unique(competencies[in_module])
    module_grade_data = np.zeros((len(module_competencies), 3))

    for i, module_competency in enumerate(module_competencies):
        specific_competency = module_competency == competencies
        for j, level in enumerate([1, 2, 3]):
            reference = (specific_competency) & (level == levels)
            if np.sum(reference) == 0:
                module_grade_data[i,j] = np.nan
            else:
                module_grade_data[i,j] = np.around(np.mean(grade[reference]), decimals=1)

    module_df = pd.DataFrame(
        module_grade_data
    )
    module_df.columns = ["Level 1", "Level 2", "Level 3"]
    module_df.index = module_competencies
    
    return module_df

In [6]:
wb = xl.Workbook(r'S:\TEAP Resources\SB TEAP Moodle Scraper\SB Official Progression Monitor Tool.xlsx')
sh = xl.Sheet("Your Points")

def input_dataframe_into_excel(df, excel_range):
    values = df.values.copy()
    values[values == 0] = None
    values[np.isnan(values)] = None
    
    xl.Range(sh, excel_range).value = values

In [7]:
module_excel_ranges = {
    2: 'C21:E26',
    3: 'C33:E37',
    4: 'C45:E51',
    5: 'C64:E68',
    6: 'C77:E83',
    1: 'C95:E97',
    7: 'C103:E106',
    8: 'C112:E118'
}

def main(module_number):
    df = create_module_dataframe(module_number)
    input_dataframe_into_excel(df, module_excel_ranges[module_number])
    
    return df

In [8]:
for module_number in [2,3,4,5,6,1,7,8]:
    df = main(module_number)
    display(df)

,Level 1,Level 2,Level 3
2.1,1,0,0
2.2,1,0,0
2.3,1,0,0
2.4,1,0,0
2.5,1,0,0
2.6,1,0,0


,Level 1,Level 2,Level 3
3.1,1,0.5,0
3.2,1,0.5,0
3.3,1,0.7,0
3.4,1,0.8,0
3.5,1,0.0,0


,Level 1,Level 2,Level 3
4.1,1,1.0,0
4.2,1,1.0,0
4.3,1,0.0,0
4.4,1,1.0,0
4.5,1,0.5,0
4.6,1,0.0,0
4.7,1,0.0,0


,Level 1,Level 2,Level 3
5.1,1.0,0.4,0
5.2,1.0,0.0,0
5.3,0.8,0.0,0
5.4,1.0,0.0,0
5.5,1.0,0.2,0


,Level 1,Level 2,Level 3
6.1,1,0.7,NaN
6.2,1,0.3,NaN
6.3,1,0.0,NaN
6.4,1,0.0,NaN
6.5,1,0.0,NaN
6.6,1,0.0,NaN
6.7,NaN,NaN,0


,Level 1,Level 2,Level 3
1.1,0.5,0,NaN
1.2,1.0,0,NaN
1.3,0.0,NaN,NaN


,Level 1,Level 2,Level 3
7.1,0,NaN,NaN
7.2,0,0,NaN
7.3,0,NaN,NaN
7.4,1,0,NaN


,Level 1,Level 2,Level 3
8.1,1,NaN,NaN
8.2,1,NaN,NaN
8.3,1,NaN,NaN
8.4,1,NaN,NaN
8.5,0,NaN,NaN
8.6,1,NaN,NaN
8.7,0,NaN,NaN
